<a href="https://colab.research.google.com/github/AVIDALESB/TelecomX-Evasion-de-Clientes-P2/blob/main/TelecomX_Evasion_de_Clientes_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Extracción del Archivo Tratado***

> Agregar bloque entrecomillado



In [1]:
import pandas as pd

In [5]:
datos = pd.read_csv('/content/df.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/df.csv'

In [ ]:
datos['género'] = datos['género'].map({'Masculino': 0, 'Femenino': 1})

In [ ]:
datos = datos.rename(columns={'cuidadano_mayor': 'ciudadano_mayor'})

In [ ]:
datos

In [ ]:
datos.info()

In [ ]:
datos.isnull().sum()

# ***Eliminación de las Columnas Irrelevantes***

In [ ]:
print(datos.nunique().sort_values())

In [ ]:
columnas_unicas = [col for col in datos.columns if datos[col].nunique() == 1]
print(f'Las columnas con valores únicos son: {columnas_unicas}')

In [ ]:
correlacion = datos.select_dtypes(include=['float64', 'int64']).corr().abs()
correlacion

In [ ]:
# Dado que cuentas_diarias y cargos_mensuales tienen la misma correlatividad, conviene eliminar cuentas_diarias.
# Otras columnas que se eliminarán serán id_cliente porque tiene identificadores únicos y cantidad_servicios por su alta correlación en columnas irrelevantes para relacionarlas con churn.
# También, otras columnas que se eliminará por su baja correlacion con churn y el resto de columnas serán servicio_telefonico y género.
# Si bien cargos_totales puede ser útil con churn, su multicolinealidad del 0.82 con meses_de_contrato la hace una columna inviable para el modelo.

In [ ]:
df_reducido = datos.drop(columns = ['id_cliente', 'cuentas_diarias', 'cantidad_servicios', 'servicio_telefonico', 'cargos_totales', 'género'])
df_reducido.info()

# ***Verificación de la Proporción de Cancelación (Churn)***

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

valores = df_reducido['churn'].value_counts()
porcentajes = df_reducido['churn'].value_counts(normalize=True) * 100

df_aux = df_reducido['churn'].map({0: 'Permanece', 1: 'Abandonó'}).to_frame(name='Estado')
sns.set_style("whitegrid")
plt.figure(figsize=(7, 7))
colores = ['#0083d5', '#d56e00']

ax = sns.countplot(x='Estado', data=df_aux, color=None)
for patch, color in zip(ax.patches, colores):
    patch.set_facecolor(color)

plt.title('PROPORCIÓN DE CANCELACIÓN (CHURN)', fontsize=14, fontweight='bold', loc='center')
plt.xlabel('ESTADO DEL CLIENTE', fontsize=10, fontweight='bold')
plt.ylabel('NÚMERO DE CLIENTES', fontsize=10, fontweight='bold')

for p in ax.patches:
    cantidad = int(p.get_height())
    porcentaje = cantidad / len(df_reducido['churn']) * 100
    ax.annotate(f'{cantidad} / {porcentaje:.1f}%',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom', fontsize=10, color='black',
                xytext=(0, 1), textcoords='offset points')

plt.tight_layout()
plt.show()

# ***Análisis de Correlación***

In [ ]:
df_corr = df_reducido.select_dtypes(include=['float64', 'int64']).corr().abs()

In [ ]:
import plotly.express as px

fig = px.imshow(
    df_corr,
    text_auto=".2f",
    color_continuous_scale="viridis",
    aspect="auto",
    title="MATRIZ DE CORRELACIÓN"
)
fig.update_layout(margin=dict(l=40, r=40, t=60, b=40))
fig.show()

In [ ]:
# Al analizar la primera columna de la matriz, se puede identificar claramente los factores más influyentes.
# El predictor principal es meses_de_contrato con una correlación del 0.34. La interpretación es que a mayor antigüedad, menor es la probabilidad de cancelar.
# Después hay predictores secundarios cuyo impacto es moderado. Por ejemplo: cargos_mensuales y factura_en_linea (ambas con 0.18) mantienen un peso considerable.
# Ambos intuyen que los clientes con cargos más altos o con factura online tienen una leve tendencia mayor a cancelar.
# El grupo de variables demográficas ahora tiene más protagonismo (dependientes con 0.15, pareja con 0.14 y ciudadano_mayor con 0.14), aunque su impacto individual es débil.
# En conjunto le dan al modelo un perfil claro del cliente. Por ejemplo: Ser un ciudadano mayor, tener pareja o dependientes tiene una influencia positiva muy leve en la probabilidad de abandono.

# Otras conclusiones a sacar son que la correlación más fuerte en toda la matriz es entre pareja y dependientes con 0.45, por lo que quienen tengan pareja son más propensos a tener dependientes.
# Otra correlación moderada es entre pareja y meses_de_contrato con 0.38, esto sugiere que los clientes con pareja tienden a tener contratos de mayor duración.
# Y por último, la correlación entre factura_en_linea y cargos_mensuales es moderada con 0.35, eso indica que los clientes con cargos mensuales más altos son más propensos a usar la facturación en línea.

# ***Análisis Dirigido***

In [ ]:
from matplotlib.lines import Line2D

sns.set(style="whitegrid")
plt.figure(figsize=(7, 6))

ax = sns.boxplot(
    data=df_reducido,
    x='churn',
    y='meses_de_contrato',
    hue='churn',
    palette={0: '#0083d5', 1: '#d56e00'},
    legend=False,
    medianprops={'linewidth': 2}
)

for i, (name, group) in enumerate(df_reducido.groupby('churn')):
    ax.hlines(
        y=group['meses_de_contrato'].mean(),
        xmin=i - 0.4,
        xmax=i + 0.4,
        color='cyan',
        linestyle='--',
        linewidth = 2
    )

plt.title("DISTRIBUCIÓN DE LOS MESES DE CONTRATO SEGÚN LA CANCELACIÓN", fontsize=14, fontweight='bold', loc = 'center')
plt.xlabel("ESTADO DEL CLIENTE", fontsize=12, fontweight='bold')
plt.ylabel("MESES DE CONTRATO", fontsize=12, fontweight='bold')

legend_elements = [
    Line2D([0], [0], color='cyan', linestyle='--', label='Media'),
    Line2D([0], [0], color='black', label='Mediana')
]

ax.legend(handles=legend_elements, loc='upper center')
plt.tight_layout()
plt.show()

In [ ]:
# En el grupo 0 la mediana es ligeramente más alta que la media (aunque ambas tienen casi el mismo valor, aproximadamente 38 meses).
# Hay una amplia dispersión en el rango de los meses (De 1 a más de 70), lo que indica una clientela con permanencias muy diversas.
# La mayoría de los clientes que no cancelaron han permanecido mucho más tiempo con la empresa.
# Mientras tanto, en el grupo 1, la mediana está entre 10 y 12 meses y la media está entre 19 y 20 meses, por lo que es más alta que la mediana.
# Los clientes que cancelan tienden a tener menos meses de contrato.

# Esto esto indica una relación entre las variables: Cuanto más meses de contrato, menor es la probabilidad de cancelación, por lo que la retención de largo plazo disminuye la tasa de churn.

In [ ]:
from matplotlib.lines import Line2D

sns.set(style="whitegrid")
plt.figure(figsize=(7, 6))

ax = sns.boxplot(
    data=df_reducido,
    x='churn',
    y='cargos_mensuales',
    hue='churn',
    palette={0: '#0083d5', 1: '#d56e00'},
    legend=False,
    medianprops={'linewidth': 2}
)

for i, (name, group) in enumerate(df_reducido.groupby('churn')):
    ax.hlines(
        y=group['cargos_mensuales'].mean(),
        xmin=i - 0.4,
        xmax=i + 0.4,
        color='cyan',
        linestyle='--',
        linewidth = 2
    )

plt.title("DISTRIBUCIÓN DE LOS CARGOS MENSUALES SEGÚN LA CANCELACIÓN", fontsize=14, fontweight='bold', loc = 'center')
plt.xlabel("ESTADO DEL CLIENTE", fontsize=12, fontweight='bold')
plt.ylabel("CARGOS MENSUALES", fontsize=12, fontweight='bold')

legend_elements = [
    Line2D([0], [0], color='cyan', linestyle='--', label='Media'),
    Line2D([0], [0], color='black', label='Mediana')
]

ax.legend(handles=legend_elements, loc='upper center')
plt.tight_layout()
plt.show()

In [ ]:
# En este grupo 0 la mediana de cargos mensuales es cercana a 65 al igual que la media, solo que esta es mínimamente menor a la mediana (aproximadamente 62).
# Existe una distribución amplia pero es relativamente más baja en valores.
# En cambio, en el grupo 1 la mediana ronda casi los 80 (es mayor que el grupo que no canceló) y la media ronda si los 75, siendo más alto que el grupo 0.
# Esto demuestra que muchos clientes que cancelan tienen cargos mensuales más altos.

# En este boxplot existe una relación inversa entre las variables: Los clientes con cargos más altos tienden a cancelar más.
# Esto podría deberse a una percepción de alto costo o baja percepción de valor por el precio pagado.

# ***Separación de los Datos***

In [ ]:
df_reducido.columns

In [ ]:
df_reducido

In [ ]:
X = df_reducido.drop('churn', axis = 1)
y = df_reducido['churn']

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify = y, random_state=42)

In [ ]:
columnas_categoricas = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
columnas_numericas = ['meses_de_contrato', 'cargos_mensuales']

# ***Encoding***

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
one_hot = make_column_transformer((OneHotEncoder(sparse_output=False, handle_unknown='ignore'), columnas_categoricas), (StandardScaler(), columnas_numericas), remainder='passthrough', sparse_threshold=0)

In [ ]:
X_train_encoded = one_hot.fit_transform(X_train)
X_test_encoded = one_hot.transform(X_test)

In [ ]:
one_hot.get_feature_names_out()

In [ ]:
X_train_df = pd.DataFrame(X_train_encoded, columns = one_hot.get_feature_names_out())
X_train_df

In [ ]:
X_train_df.info()

In [ ]:
X_test_df = pd.DataFrame(X_test_encoded, columns=one_hot.get_feature_names_out())
X_test_df

In [ ]:
X_test_df.info()

In [ ]:
X_train_corr = X_train_df.copy()
X_train_corr['churn'] = y_train.reset_index(drop=True)

correlaciones = X_train_corr.corr()['churn'].sort_values(ascending=False)
print(correlaciones)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV

modelo_l1 = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
selector = RFECV(estimator=modelo_l1, step=1, cv=5, scoring='accuracy')
selector.fit(X_train_df, y_train)

selected_features = X_train_df.columns[selector.support_]
print("Número óptimo de variables:", selector.n_features_)

In [ ]:
from sklearn.feature_selection import RFE

modelo_base = LogisticRegression(penalty='l1', solver='liblinear', C = 10, random_state=42)
selector = RFE(estimator=modelo_base, n_features_to_select=29)

selector.fit(X_train_df, y_train)
selected_features_mask = selector.support_
selected_features = X_train_df.columns[selected_features_mask]

X_train_selected = X_train_df[selected_features]
X_test_selected = X_test_df[selected_features]

print("Las variables seleccionadas son:")
print(selected_features)

# ***Balanceo de las Clases***

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(random_state=42)
X_train_bal, y_train_bal = ros.fit_resample(X_train_selected, y_train)

print("Distribución original:", Counter(y))
print("Distribución del entrenamiento antes del balanceo:", Counter(y_train))
print("Distribución del entrenamiento balanceado:", Counter(y_train_bal))

counter = Counter(y_train_bal)
if len(set(counter.values())) == 1:
    print("Las clases están perfectamente balanceadas.")
else:
    print("Las clases NO están balanceadas perfectamente.")

# ***Normalización o Estandarización***

In [ ]:
# En esta etapa, no hace falta normalizar o estandarizar los datos antes del entrenamiento de modelos ya que si bien hay algunos que lo necesitan.
# La realidad es que ya se aplicó la estandarización automáticamente a las variables numéricas mediante StandardScaler dentro del make_column_transformer.
# Eso si, las variables binarias ('ciudadano_mayor', 'pareja', 'dependientes' y 'factura_en_linea') no se estandarizaron debido a que su escala no afecta a los algoritmos

# ***Creación y Evaluación de los Modelos***

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def evaluar_modelo(y_test, y_pred):
  print("Resultados para el modelo:")
  print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
  print(f"Precision: {precision_score(y_test, y_pred):.2f}")
  print(f"Recall: {recall_score(y_test, y_pred):.2f}")
  print(f"F1: {f1_score(y_test, y_pred):.2f}")

In [ ]:
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(
    penalty='l2',
    C=10,
    class_weight= 'balanced',
    solver='liblinear',
    max_iter=5000,
    random_state=42
)

modelo.fit(X_train_bal, y_train_bal)
y_modelo_lr = modelo.predict(X_test_selected)
evaluar_modelo(y_test, y_modelo_lr)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo2 = RandomForestClassifier(
    n_estimators=50,
    max_depth=100,
    min_samples_split=5,
    min_samples_leaf=10,
    max_features='log2',
    class_weight='balanced',
    random_state=42
)

modelo2.fit(X_train_bal, y_train_bal)
y_modelo_rf = modelo2.predict(X_test_selected)
evaluar_modelo(y_test, y_modelo_rf)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

modelo3 = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

modelo3.fit(X_train_bal, y_train_bal)
y_modelo_gb = modelo3.predict(X_test_selected)
evaluar_modelo(y_test, y_modelo_gb)

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import numpy as np

modelo_rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=100,
    min_samples_split=5,
    min_samples_leaf=10,
    max_features='log2',
    class_weight='balanced',
    random_state=42
)

modelo_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    random_state=42
)

modelo_voto = VotingClassifier(estimators=[('rf', modelo_rf), ('gb', modelo_gb)],voting='soft')
modelo_voto.fit(X_train_bal, y_train_bal)
y_proba = modelo_voto.predict_proba(X_test_selected)[:, 1]
umbral = 0.45
y_pred = (y_proba >= umbral).astype(int)
evaluar_modelo(y_test, y_pred)

In [ ]:
from sklearn.metrics import RocCurveDisplay
RocCurveDisplay.from_predictions(y_test, y_pred, name = 'Modelo Híbrido');

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

matriz_confusion = confusion_matrix(y_test, y_pred)
visualizacion_matriz = ConfusionMatrixDisplay(confusion_matrix = matriz_confusion, display_labels = ['Se quedó', 'Abandonó'])
visualizacion_matriz.plot();

# ***Análisis de la Importancia de las Variables según los Modelos***

In [ ]:
rf_importance = modelo2.feature_importances_
gb_importance = modelo3.feature_importances_

feature_names = X_train_selected.columns
importance_df = pd.DataFrame({
    'Variable': feature_names,
    'RandomForest': rf_importance,
    'GBM': gb_importance
})

fig, axes = plt.subplots(1, 2, figsize=(20, 8))
rf_plot_data = importance_df.sort_values('RandomForest', ascending=False).head(15)
sns.barplot(x='RandomForest', y='Variable', data=rf_plot_data, ax=axes[0], palette='viridis', hue='Variable', legend=False)
axes[0].set_title('IMPORTANCIA DE LAS VARIABLES - RANDOM FOREST', fontsize=14, fontweight='bold')
axes[0].set_xlabel('IMPORTANCIA', fontsize=10, fontweight='bold')
axes[0].set_ylabel('VARIABLES', fontsize=10, fontweight='bold')

gbm_plot_data = importance_df.sort_values('GBM', ascending=False).head(15)
sns.barplot(x='GBM', y='Variable', data=gbm_plot_data, ax=axes[1], palette='plasma', hue='Variable', legend=False)
axes[1].set_title('IMPORTANCIA DE LAS VARIABLES - GBM', fontsize=14, fontweight='bold')
axes[1].set_xlabel('IMPORTANCIA', fontsize=10, fontweight='bold')
axes[1].set_ylabel('VARIABLES', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
comparison_table = importance_df.sort_values(by='RandomForest', ascending=False).reset_index(drop=True).head(15)
print("TABLA COMPARATIVA DE LA IMPORTANCIA DE LAS VARIABLES")
print("-" * 50)
print(comparison_table)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

features_to_plot = [
    'standardscaler__meses_de_contrato',
    'standardscaler__cargos_mensuales',
    'onehotencoder__tipo_de_contrato_Mes a mes',
    'onehotencoder__forma_de_pago_Cheque electrónico',
    'onehotencoder__tipo_de_contrato_Dos años',
    'onehotencoder__servicio_internet_Fibra óptica'
]

fig, axs = plt.subplots(2, 3, figsize=(30, 10))
palette = sns.color_palette("Set2", len(features_to_plot))
line_kw_list = [{"color": c, "linewidth": 2, "linestyle": "--"} for c in palette]
fig.suptitle('GRÁFICOS DE DEPENDENCIA PARCIAL', fontsize=18, fontweight='bold')

for i, feature in enumerate(features_to_plot):
    ax = axs.ravel()[i]

    PartialDependenceDisplay.from_estimator(
        modelo3,
        X_train_selected,
        [feature],
        ax=ax,
        kind='average',
        grid_resolution=100,
        line_kw=line_kw_list[i]
    )

    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# ***Conclusión Final***

🔹 Introducción:

Este informe enmarca la 2da parte del proyecto de Telecom X, una empresa que enfrenta una alarmante tasa de cancelaciones de su clientela. El objetivo principal fue diseñar modelos predictivos, realizar un análisis de los factores que más influyen en la cancelación y emprender acciones para mejorar la retención de clientes.

El proyecto se desarrolló a partir del DataFrame ya procesado en la primera parte, incluyendo la separación de datos, codificación, balanceo de clases, y la posterior creación y evaluación de los modelos.

---

🔹 Objetivos:

Debido a que la adquisición de nuevos clientes es significativamente más costosa que la retención de los existentes, se ha llevado a cabo un análisis de para construir modelos predictivos que permitan:

1. Identificar y cuantificar los principales factores que conducen a la cancelación.
2. Minimizar la pérdida de clientes.
3. Entender el comportamiento diferencial de los modelos predictivos.
4. Proponer un conjunto de estrategias de retención para mitigar los riesgos identificados.

---

🔹 Metodología:

Se entrenaron 3 modelos de clasificación reconocidos por su alto rendimiento, los cuales son Logistic Regression (LR), Random Forest (RF), Gradient Boosting Machine (GBM). Debido al primer modelo que se usó, fue necesario normalizar los datos en el encoding mediante el StandardScaler(). A continuación, se detalla su impacto en los 3 modelos:

- La Regresión Logística calcula una función lineal de los predictores para estimar probabilidades. Si las variables tienen escalas muy distintas, aquellas con escalas mayores tienden a dominar la magnitud de los coeficientes, afectando negativamente la convergencia del algoritmo de optimización. Los beneficios de normalizar son: una mejora en la estabilidad numérica, una aceleración en el proceso de entrenamiento y permite una comparación más justa entre coeficientes (aunque no directamente interpretable como importancia).

- El Random Forest y Gradiente Boosting Machine no requieren normalización ya que construyen árboles de decisión que dividen los datos según umbrales en variables individuales. Este proceso no depende de las distancias ni de la magnitud relativa de las variables, sino de su capacidad para dividir los datos de manera informativa.

De estos 3 modelos, el Gradient Boostin Machine presentó las mejores métricas individualmente, no obstante, para obtener un leve aumento en las estadísticas, se decidió usar un modelo de ensamble mediante VotingClassifier(). Con este método se emplearon los modelos Random Forest (bagging) y Gradient Boosting Machine (boosting). Esta combinación permite capturar distintas formas de aprender patrones en los datos, potenciando la capacidad de generalización del modelo final.

---

🔹 Análisis de los factores clave de cancelación:

El análisis de los gráficos reveló patrones claros que los modelos aprendieron para predecir el churn. ***La conclusión principal es que los factores contractuales son, por amplio margen, los predictores más potentes y fiables. Otros factores, como el método de pago y el tipo de servicio, aportan información secundaria pero comercialmente relevante***.

La evidencia más contundente se encuentra en las variables relacionadas con la duración del contrato. El gráfico de meses contratados muestra una relación inversa y casi lineal: A mayor duración, menor probabilidad de cancelación. Esta tendencia se refuerza con los gráficos categóricos, donde tener un contrato "Mes a Mes" incrementa significativamente el riesgo de churn, mientras que los contratos de "Dos Años" lo reducen. Así, los tres gráficos narran una historia coherente: *La estabilidad contractual es el pilarde la retención de clientes según los modelos*.

Más allá de los contratos, emergen dos factores relevantes. Primero, **el uso del cheque electrónico como método de pago se asocia con una mayor probabilidad de cancelación**. Esto podría deberse a que es un medio menos automatizado o a que refleja un perfil de cliente con menor estabilidad financiera o digital.

En segundo lugar, de forma contraintuitiva, **ser cliente de fibra óptica también incrementa el riesgo**. Este hallazgo sugiere que el servicio premium no cumple con las expectativas, presenta problemas de fiabilidad o tiene un precio percibido como injusto, lo que eleva la probabilidad de churn.

Por último, **los cargos mensuales presentan un comportamiento más complejo**. A diferencia de otras variables, no muestran una tendencia clara: su relación con la cancelación es volátil y depende de interacciones con otros factores. Por ejemplo, un precio alto puede ser aceptable en un contrato largo, pero riesgoso en uno de mes a mes. Como los modelos captaron esta complejidad, una política de precios simple no será efectiva, su impacto debe analizarse en función del perfil completo del cliente.

De este modo, se identifican tres áreas principales que impulsan la cancelación:

---

*1. Factor dominante: La estabilidad contractual (gráficos PDP y de correlación)*

- **La duración y el tipo de contrato son los predictores más potentes** según los modelos Random Forest y GBM.

- **El contrato "Mes a Mes" representa el principal indicador de riesgo**. Los gráficos PDP muestran que este tipo de contrato eleva notablemente la probabilidad de cancelación, y el modelo GBM le asigna una importancia crítica.

- En contraste, **la duración del contrato actúa como el principal factor de retención**. Los gráficos de correlación y el boxplot indican que a mayor cantidad de meses contratados, menor es el riesgo. Los clientes con contrato de 2 años son los más leales.

---

*2. Factores económicos y de conveniencia (gráficos PDP y boxplot)*

- **El pago mediante cheque electrónico se asocia con mayor probabilidad de churn**, posiblemente por ser un método menos automático o un indicador de menor estabilidad del cliente.

- **Los cargos mensuales no presentan una relación lineal clara** en los gráficos PDP, pero el boxplot muestra que los clientes con cargos más altos tienden a cancelar más. Por lo que este efecto depende del contexto: Un precio elevado puede ser aceptables si está respaldado por contratos largos o servicios de valor significantes.

---

*3. Calidad percibida del servicio (gráfico PDP)*

- De manera sorprendente, **ser cliente de fibra óptica se vincula con mayor riesgo de cancelación**. Esto podría deberse a una brecha entre las expectativas creadas y la experiencia real, precios poco competitivos o deficiencias técnicas.

---

🔹 Conclusión:

Tanto los modelos predictivos como el modelo híbrido han proporcionado una hoja de ruta clara para enfocar los esfuerzos de retención. Ahora se sabe que **la batalla por la retención se gana en la estructura del contrato**. Al enfocar los recursos en migrar a los clientes de alto riesgo a planes más estables y en investigar las deficiencias operativas de servicios claves, la empresa puede construir una base de clientes más leal y reducir significativamente la tasa de cancelación a largo plazo.

Para complementar estas decisiones se utilizó Feature Importance para analizar la importancia de las variables de mayor a menor influencia en la predicción, Gráficos de Dependencia Parcial (PDP) para visualizar cómo afecta el cambio en una variable específica a la probabilidad de cancelación y entender la dirección y magnitud del efecto.

Eso sí, aunque el modelo de Random Forest y GMB tuvieron diferencias en la distribución de la importancia de las variables, *ambos modelos mostraron un alto grado de acuerdo en la identificación de las variables más importantes*, lo que refuerza la validez de los hallazgos.

---

🔹 Recomendaciones:

Tras mi conclusión y análisis, recomiendo las siguientes estrategias:

1. ***Fortalecer el compromiso contractual***. El objetivo es reducir la cantidad de clientes con contrato "Mes a mes". Para lograrlo, la empresa debería implementar campañas proactivas dirigidas a quienes ya llevan más de 3 a 6 meses bajo esta modalidad, ofreciéndoles incentivos concretos para migrar a contratos de 1 o 2 años, como descuentos en la tarifa mensual, un mes gratuito o servicios adicionales sin cargo. Además, se recomienda utilizar el modelo predictivo para identificar y priorizar a los clientes con mayor probabilidad de cancelar.

2. ***Optimizar la experiencia de pago***. El objetivo es reducir la fricción y aumentar la retención entre quienes utilizan cheque electrónico. Para ello, sugiero lanzar una campaña preventiva que comunique los beneficios de los métodos automáticos como tarjeta de crédito o débito, acompañada de un incentivo menor (como un descuento único) para quienes realicen el cambio. También es clave analizar las tasas de fallo de este medio de pago para identificar si las cancelaciones se deben a problemas operativos.

3. ***Auditar y mejorar los servicios, especialmente la fibra óptica***. Es necesario comprender por qué un producto considerado premium se asocia a una mayor tasa de cancelación. Recomiendo lanzar encuestas de satisfacción específicas para los usuarios de fibra óptica y demás servicios y revisar los registros del centro de atención al cliente para detectar patrones de queja (velocidad, interrupciones, instalación). Por último, conviene realizar un benchmark de precios y rendimiento frente a competidores en las zonas con cobertura y otros servicios.


---

🔹 Final:

Si bien, todo el análisis ETL permitió sentar las bases para un sistema de predicción, una optimización de ofertas y una retención de clientes. Ahora, la empresa de Telecom X tiene todas las herramientas necesarias para reducir el churn.